### NLP pre-processing

Import and load data

In [44]:
import pandas as pd
import unicodedata
import contractions
import re
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df_train = pd.read_csv('train.csv')
df_test= pd.read_csv('test.csv')


In [3]:
df_pp = pd.read_csv('train.csv')

This was needed because the first row was set as the columns names. I will change the read_csv to automatically set new column names

In [4]:
wrong_train_columns = dict()
wrong_train_columns['rating'] = df_pp.columns[0]
wrong_train_columns['title'] = df_pp.columns[1]
wrong_train_columns['text'] = df_pp.columns[2]
df_pp.columns= ['rating','title','text']
df_pp.append(wrong_train_columns,ignore_index=True)

,rating,title,text
0,5,Inspiring,I hope a lot of people hear this cd. We need m...
1,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
3,5,Too good to be true,Probably the greatest soundtrack in history! U...
4,5,There's a reason for the price,"There's a reason this CD is so expensive, even..."
5,1,Buyer beware,"This is a self-published book, and if you want..."
6,4,"Errors, but great story",I was a dissapointed to see errors on the back...
7,1,The Worst!,A complete waste of time. Typographical errors...
8,1,Oh please,I guess you have to be a romance novel lover f...
9,1,Awful beyond belief!,I feel I have to write to keep others from was...


In [5]:
df_train.head()

,3,more like funchuck,"Gave this to my dad for a gag gift after directing ""Nunsense,"" he got a reall kick out of it!"
0,5,Inspiring,I hope a lot of people hear this cd. We need m...
1,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
3,5,Too good to be true,Probably the greatest soundtrack in history! U...
4,5,There's a reason for the price,"There's a reason this CD is so expensive, even..."


In [6]:
wrong_train_columns = dict()
wrong_train_columns['rating'] = df_train.columns[0]
wrong_train_columns['title'] = df_train.columns[1]
wrong_train_columns['text'] = df_train.columns[2]
df_train.columns= ['rating','title','text']
df_train.append(wrong_train_columns,ignore_index=True)

,rating,title,text
0,5,Inspiring,I hope a lot of people hear this cd. We need m...
1,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
3,5,Too good to be true,Probably the greatest soundtrack in history! U...
4,5,There's a reason for the price,"There's a reason this CD is so expensive, even..."
5,1,Buyer beware,"This is a self-published book, and if you want..."
6,4,"Errors, but great story",I was a dissapointed to see errors on the back...
7,1,The Worst!,A complete waste of time. Typographical errors...
8,1,Oh please,I guess you have to be a romance novel lover f...
9,1,Awful beyond belief!,I feel I have to write to keep others from was...


In [7]:
wrong_test_columns = dict()
wrong_test_columns['rating'] = df_test.columns[0]
wrong_test_columns['title'] = df_test.columns[1]
wrong_test_columns['text'] = df_test.columns[2]
df_test.columns= ['rating','title','text']
df_test.append(wrong_test_columns,ignore_index=True)

,rating,title,text
0,4,Surprisingly delightful,This is a fast read filled with unexpected hum...
1,2,"Works, but not as advertised",I bought one of these chargers..the instructio...
2,2,Oh dear,I was excited to find a book ostensibly about ...
3,2,Incorrect disc!,"I am a big JVC fan, but I do not like this mod..."
4,2,Incorrect Disc,"I love the style of this, but after a couple y..."
5,2,DVD menu select problems,I cannot scroll through a DVD menu that is set...
6,3,My 2 y/o grandson loves it!!,This movie with all of its animals really keep...
7,5,A Cookbook Every Baker Should Own,I found a copy of this cookbook at a local use...
8,3,good basic,"The book is a basic ""how to"" book for using so..."
9,3,nice screen for a nice price but....,I compared a few different flat panels with re...


I will only use rating and text at first because there were missing titles and one worded titles

In [10]:
df_train = df_train[['rating','text']]
df_test = df_test[['rating','text']]

Changed the rating from 0 to 5 to good, bad or neutral. Not sure if it was the best approach though

In [18]:
def good_bad_neutral(number):
    if number > 3:
        return 'good'
    elif number == 3:
        return 'neutral'
    else:
        return 'bad'

In [ ]:
pattern = r'[^a-zA-Z\s]'
stop_words = set(stopwords.words('english'))
porter = PorterStemmer()

Removing accented words, contractions, special characters, multiple whitespaces from text and making it lowercase

In [13]:
df_train.text = df_train.text.apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore'))
df_train.text = df_train.text.apply(lambda x: contractions.fix(x))
df_train.text = df_train.text.apply(lambda x:re.sub(pattern, '', x))
df_train.text = df_train.text.apply(lambda x: x.lower())
df_train.text = df_train.text.apply(lambda x: strip_multiple_whitespaces(x))

In [19]:
df_train.rating = df_train.rating.apply(lambda x: good_bad_neutral(x))
df_train.text = df_train.text.apply(lambda x: word_tokenize(x))
df_train.text = df_train.text.apply(lambda x: [word for word in x if word not in stop_words])
df_train.text = df_train.text.apply(lambda x: [porter.stem(word) for word in x])

In [50]:
df_train.head()

,rating,text
0,good,"[hope, lot, peopl, hear, cd, need, strong, pos..."
1,good,"[read, lot, review, say, best, game, soundtrac..."
2,good,"[music, yasunori, misuda, without, question, c..."
3,good,"[probabl, greatest, soundtrack, histori, usual..."
4,good,"[reason, cd, expens, even, version, importsom,..."


I'm not sure if it necessary to join the document or having a list of the words is enough

In [51]:
df_train_joined = df_train
df_train_joined.text = df_train_joined.text.apply(lambda x: ' '.join(x))
df_train_joined.head()

,rating,text
0,good,hope lot peopl hear cd need strong posit vibe ...
1,good,read lot review say best game soundtrack figur...
2,good,music yasunori misuda without question close s...
3,good,probabl greatest soundtrack histori usual bett...
4,good,reason cd expens even version importsom best m...


In [60]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_train_joined.text)

In [36]:
df_test.text = df_test.text.apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore'))
df_test.text = df_test.text.apply(lambda x: contractions.fix(x))
pattern = r'[^a-zA-Z\s]'
df_test.text = df_test.text.apply(lambda x:re.sub(pattern, '', x))
df_test.text = df_test.text.apply(lambda x: x.lower())
df_test.text = df_test.text.apply(lambda x: strip_multiple_whitespaces(x))

In [37]:
df_test.rating = df_test.rating.apply(lambda x: good_bad_neutral(x))
df_test.text = df_test.text.apply(lambda x: word_tokenize(x))
df_test.text = df_test.text.apply(lambda x: [word for word in x if word not in stop_words])
df_test.text = df_test.text.apply(lambda x: [porter.stem(word) for word in x])

In [61]:
df_test_joined = df_test
df_test_joined.text = df_test_joined.text.apply(lambda x: ' '.join(x))
df_test_joined.head()

,rating,text
0,good,fast read fill unexpect humour profound insigh...
1,bad,bought one chargersth instruct say light stay ...
2,bad,excit find book ostens muslim femin volum live...
3,bad,big jvc fan like model suspisci saw sever unit...
4,bad,love style coupl year dvd give problem even wo...


In [62]:
X_test = vectorizer.fit_transform(df_test_joined.text)